## Support Vector Regression

### Creating Dataset ( Tips Dataset )

In [2]:
import seaborn as sns

df = sns.load_dataset("tips")

In [5]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [6]:
df['sex'].value_counts() # Binary Category

sex
Male      157
Female     87
Name: count, dtype: int64

In [7]:
df['smoker'].value_counts() # Binary Category

smoker
No     151
Yes     93
Name: count, dtype: int64

In [8]:
df['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [9]:
df['time'].value_counts() # Binary Category

time
Dinner    176
Lunch      68
Name: count, dtype: int64

#### Independent and Dependent Features

In [10]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [11]:
X = df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y = df['total_bill']

### Split

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

In [14]:
X_train

,tip,sex,smoker,day,time,size
58,1.76,Male,Yes,Sat,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
68,2.01,Male,No,Sat,Dinner,2
184,3.00,Male,Yes,Sun,Dinner,2
...,...,...,...,...,...,...
64,2.64,Male,No,Sat,Dinner,3
15,3.92,Male,No,Sun,Dinner,2
228,2.72,Male,No,Sat,Dinner,2
125,4.20,Female,No,Thur,Lunch,6


### Feature Encoding (Label Encoding and One Hot Encoding)
- Replace Binary categories as 0's and 1's
- Where multiple category will apply one hot hotencoding

#### For Binary Categories use Label Encoder

In [15]:
from sklearn.preprocessing import LabelEncoder

##### Three Binary Categories: Create three encoders

##### Train

In [16]:
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

In [17]:
X_train['sex'] = le1.fit_transform(X_train['sex'])
X_train['smoker'] = le2.fit_transform(X_train['smoker'])
X_train['time'] = le3.fit_transform(X_train['time'])

In [18]:
X_train.head()

,tip,sex,smoker,day,time,size
58,1.76,1,1,Sat,0,2
1,1.66,1,0,Sun,0,3
2,3.50,1,0,Sun,0,3
68,2.01,1,0,Sat,0,2
184,3.00,1,1,Sun,0,2


In [30]:
X_train.columns

Index(['tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

##### Test

In [19]:
X_test['sex'] = le1.transform(X_test['sex'])
X_test['smoker'] = le2.transform(X_test['smoker'])
X_test['time'] = le3.transform(X_test['time'])

In [20]:
X_test.head()

,tip,sex,smoker,day,time,size
162,2.00,0,0,Sun,0,3
60,3.21,1,1,Sat,0,2
61,2.00,1,1,Sat,0,2
63,3.76,1,1,Sat,0,4
69,2.09,1,1,Sat,0,2


#### One hot encoding --- columnTransformer

In [21]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

### Understanding One-Hot Encoding with ColumnTransformer

<h3>1. Why was Thursday dropped? What about Friday?</h3>
<p>You're using:</p>
<pre>
OneHotEncoder(drop='first')
</pre>
<p>This automatically drops the first category (alphabetically) to prevent the <b>dummy variable trap</b> (redundant features in one-hot encoding).</p>

<h3>Step-by-Step Explanation</h3>
<ol>
  <li>The column <code>day</code> has categories: <b>Thur, Fri, Sat, Sun</b>.</li>
  <li>The <code>drop='first'</code> option removes the first category (<b>Thur</b>).</li>
  <li>The remaining categories (<b>Fri, Sat, Sun</b>) are one-hot encoded.</li>
</ol>

<p>Resulting one-hot encoding:</p>
<table>
  <tr>
    <th>Original 'day'</th>
    <th>day_Fri</th>
    <th>day_Sat</th>
    <th>day_Sun</th>
  </tr>
  <tr>
    <td>Thur</td>
    <td>0</td>
    <td>0</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Fri</td>
    <td>1</td>
    <td>0</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Sat</td>
    <td>0</td>
    <td>1</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Sun</td>
    <td>0</td>
    <td>0</td>
    <td>1</td>
  </tr>
</table>

<h3>2. Why does <code>X_train[0]</code> give <code>[1. , 0. , 0. , 1.76, 1. , 1. , 0. , 2. ]</code>?</h3>
<p>After applying <code>ct.fit_transform(X_train)</code>, each row gets transformed.</p>

<h4>Step-by-Step Explanation</h4>
<ol>
  <li>The <b>one-hot encoding</b> replaces the <code>day</code> column with <code>day_Fri</code>, <code>day_Sat</code>, and <code>day_Sun</code>.</li>
  <li>The remaining columns <b>(tip, sex, smoker, time, size)</b> stay unchanged.</li>
  <li>The first row of the transformed dataset is: <code>[1. , 0. , 0. , 1.76, 1. , 1. , 0. , 2. ]</code>.</li>
</ol>

<p>Breakdown of <code>[1. , 0. , 0. , 1.76, 1. , 1. , 0. , 2. ]</code>:</p>
<table>
  <tr>
    <th>Column</th>
    <th>Value</th>
    <th>Meaning</th>
  </tr>
  <tr>
    <td>day_Fri</td>
    <td>1</td>
    <td>Friday (since 'Thur' is dropped)</td>
  </tr>
  <tr>
    <td>day_Sat</td>
    <td>0</td>
    <td>Not Saturday</td>
  </tr>
  <tr>
    <td>day_Sun</td>
    <td>0</td>
    <td>Not Sunday</td>
  </tr>
  <tr>
    <td>tip</td>
    <td>1.76</td>
    <td>Tip amount</td>
  </tr>
  <tr>
    <td>sex</td>
    <td>1</td>
    <td>Possibly Male (assuming Male = 1, Female = 0)</td>
  </tr>
  <tr>
    <td>smoker</td>
    <td>1</td>
    <td>Smoker (Yes = 1, No = 0)</td>
  </tr>
  <tr>
    <td>time</td>
    <td>0</td>
    <td>Possibly Lunch (Lunch = 0, Dinner = 1)</td>
  </tr>
  <tr>
    <td>size</td>
    <td>2</td>
    <td>Number of people at the table</td>
  </tr>
</table>

<h5>Final Summary</h5>
<ul>
  <li><b>'Thur'</b> is dropped because <code>drop='first'</code> removes the first category.</li>
  <li>The dataset now has <b>'Fri', 'Sat', 'Sun'</b> as one-hot encoded columns.</li>
  <li>Numerical columns <b>(tip, size)</b> remain unchanged.</li>
  <li>The transformed row <code>[1. , 0. , 0. , 1.76, 1. , 1. , 0. , 2. ]</code> represents a sample from the dataset.</li>
</ul>


In [25]:
ct = ColumnTransformer(transformers=[('onehot', OneHotEncoder(drop='first'), [3])], remainder='passthrough')

In [28]:
import sys
import numpy

numpy.set_printoptions(threshold=sys.maxsize)
ct.fit_transform(X_train)[0]

array([1.  , 0.  , 0.  , 1.76, 1.  , 1.  , 0.  , 2.  ])

In [31]:
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

### SVR

In [32]:
from sklearn.svm import SVR

svr = SVR()

In [33]:
svr.fit(X_train, y_train)

SVR()

In [36]:
y_pred = svr.predict(X_test)

In [37]:
from sklearn.metrics import r2_score, mean_absolute_error
print(r2_score(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))

0.46028114561159283
4.1486423210190235


### Hyperparameter Tuning

In [38]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10, 100, 1000],
             'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
             'kernel': ['rbf']}

In [39]:
grid = GridSearchCV(SVR(), param_grid=param_grid, refit=True, verbose=3)

grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.067 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.058 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.145 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.025 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.089 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.013 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.021 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.010 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.124 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.050 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.053 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=0.01, kernel=rbf;

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [40]:
grid.best_params_

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

In [41]:
grid_predictions = grid.predict(X_test)

In [42]:
print(r2_score(y_test, grid_predictions))
print(mean_absolute_error(y_test, grid_predictions))

0.5081618245078687
3.8685147526100234
